In [34]:
import sys, os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, SimpleRNN, LSTM
from keras.models import Sequential
from tslearn.preprocessing import TimeSeriesScalerMinMax
sys.path.append(os.path.join(os.path.dirname('Demand'), '..', 'src'))
import Demand

In [2]:
path = '../data/demand_lower_48'

In [3]:
national_demand = Demand.Demand()

In [4]:
national_demand.load_and_clean_data(path)

In [5]:
national_demand.dataframe.head()

,Megawatthours,Year,Month,Hour,Day_of_week,Day_of_month,Day_of_year
Time,,,,,,,
2015-07-01 02:00:00,335153,2015,7,2,2,1,182
2015-07-01 03:00:00,333837,2015,7,3,2,1,182
2015-07-01 04:00:00,398386,2015,7,4,2,1,182
2015-07-01 05:00:00,388954,2015,7,5,2,1,182
2015-07-01 06:00:00,392487,2015,7,6,2,1,182


In [6]:
df = pd.DataFrame(national_demand.dataframe['Megawatthours'])

In [7]:
df.head()

,Megawatthours
Time,
2015-07-01 02:00:00,335153
2015-07-01 03:00:00,333837
2015-07-01 04:00:00,398386
2015-07-01 05:00:00,388954
2015-07-01 06:00:00,392487


In [8]:
split_date = '2020-07-01 00:00:00'


In [9]:
def get_split_index(df, split_date):
    df_reset = df.reset_index()
    split_index = df_reset.loc[df_reset['Time'] == split_date].index[0]
    return split_index

In [10]:
split_index = get_split_index(df, split_date)
split_index

43846

In [11]:
scaler = MinMaxScaler()

In [12]:
def create_X_matrix(df, seq_len):
    X = []
    for i in range(seq_len, len(df)):
        X.append(df['Megawatthours'].iloc[i-seq_len:i].values)
    return np.array(X)
        

In [13]:
seq_len = 20

In [14]:
X = create_X_matrix(df, seq_len)

In [19]:
X

array([[335153, 333837, 398386, ..., 586317, 572656, 554682],
       [333837, 398386, 388954, ..., 572656, 554682, 543332],
       [398386, 388954, 392487, ..., 554682, 543332, 520871],
       ...,
       [402463, 399263, 389826, ..., 366173, 363364, 365016],
       [399263, 389826, 377119, ..., 363364, 365016, 371950],
       [389826, 377119, 366604, ..., 365016, 371950, 379932]])

In [15]:
X.shape

(50159, 20)

In [16]:
y = np.array(df['Megawatthours'][seq_len:])

In [17]:
y

array([543332, 520871, 485722, ..., 371950, 379932, 390453])

In [18]:
y.shape

(50159,)

In [20]:
def split_train_test(X, y, split_index):
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    return X_train, X_test, y_train, y_test

In [21]:
X_train, X_test, y_train, y_test = split_train_test(X, y, split_index)

In [22]:
len(X_train)

43846

In [23]:
len(y_train)

43846

In [24]:
len(X_test)

6313

In [25]:
len(y_test)

6313

In [26]:
X_test[0]

array([517364, 484279, 454338, 431367, 415298, 406922, 407320, 415012,
       430991, 454903, 481534, 511932, 541324, 567029, 587465, 603816,
       615398, 623724, 627955, 623320])

In [27]:
y_test[0]

611250

In [28]:
X_train_scaled, y_train_scaled = scaler.fit_transform(X_train, y_train)

ValueError: too many values to unpack (expected 2)